In [5]:
import pandas as pd
import numpy as np
import glob
from OrderNpersist import Persist_tracks,Order_tracks2,order_radius,to_vecs
from analyse_tracks import new_auto
from numpy.linalg import norm
from itertools import product

In [6]:
files = glob.glob('../data/real_cells/exp*.txt')
# goal : 
# speed , persist, order, local order per video
# std of all these measures : 

In [7]:
# for filename in files: 
#     # extract tracks : 
#     df = pd.read_csv(filename, sep="\t", names=['cell_id', 'time_id', 'time', 'x', 'y', 'z'])   
#     df['experiment'] = filename[19:28]
#     df['cell_type'] = filename[29:32]
#     df['movie'] = filename[38:39]
#     ids = df.cell_id.unique()
#     tracks = [df[df.cell_id == i][['x','y','z']].to_numpy() for i in ids]
#     # filter out super short tracks: 
#     tracks = [t for t in tracks if len(t) > 50]
    


In [8]:
x = np.array([4,5,6])
np.sum([x,x])

30

In [9]:
from collections import defaultdict

"""Accepts a python list of 3D spatial points, e.g. [[x1,y1,z1],...],
optionally with weights e.g. [[x1,x2,x3,w1],...], and returns the sparse
histogram (i.e. no empty bins) with bins of resolution (spacing) given by
res.
The weights option allows you to chose to histogram over counts
instead of weights (equivalent to all weights being 1).
The bin_index option lets you return the points with their bin indices 
(the integers representing how many bins in each direction to walk to 
find the specified bin) rather than centerpoint coordinates."""
def sparse_hist(points, res, weights=True, bin_index=False):
    def _binindex(point):
        point = point[0]
        bi = [int(x//res) for x in point]
        bi = tuple(bi)
        return bi

    def _bincenter(point):
        point = point[0]
        bc = [(x//res+0.5)*res for x in point]
        bc = tuple(bc)
        return bc

    if not bin_index:
        if weights:
            pointlist = [(_bincenter(x), x[1]) for x in points]
        else:
            pointlist = [(_bincenter(x), 1) for x in points]
    else:
        if weights:
            pointlist = [(_binindex(x), x[1]) for x in points]
        else:
            pointlist = [(_binindex(x), 1) for x in points]

    pointdict = defaultdict(list)
    for k,v in pointlist:
        pointdict[k].append(v)

    for key,val in pointdict.items():
        val = sum(val)
        pointdict.update({key:val})

    return pointdict

In [48]:
def Global_order(vec_tracks):
    # sum vec instead of calculating angles : 
    orders = 0
    for vtrack in vec_tracks:
        for v in vtrack:
            orders += v
    return norm(orders)

def OrderAt_T(vec_tracks):
    # find smallest track : 
    min_length = min([len(t) for t in vec_tracks])
    
    # reshape vectors to square matrix : 
    vt2 = np.zeros((len(vec_tracks),min_length,3))
    for i,v in enumerate(vec_tracks):
        vt2[i] = v[:min_length]
                   
    print(vt2.shape)
    orders = []
    for i in range(min_length):
        vecs_at_t = vt2[:,i]
        ordr = 0
        for v in vecs_at_t:
            ordr += v
        orders.append(ordr)
    print('lenght of order shoulde be min_lenght. Min_lenght',min_length,'==',len(orders))
    av_ordr = np.average([norm(v) for v in orders])
    std_ordr = np.std([norm(v) for v in orders])
        
    return av_ordr,std_ordr

def local_order(tracks,bins = 5):
    # find smallest track : 
    min_length = min([len(t) for t in vec_tracks])
    flat_tracks = [item for sublist in tracks for item in sublist.flatten()]
    max_d = max(flat_tracks) # end of domain : 
    # reshape vectors to square matrix : 
    vt2 = np.zeros((len(vec_tracks),min_length,3))
    for i,v in enumerate(vec_tracks):
        vt2[i] = v[:min_length]
    
    # go through vectors per timestep :
    orders = []
    for i in range(min_length):
        # bin according to position :
        vecs_at_t = vt2[:,i]
        pos = [tracks[j][i] for j in range(len(vecs_at_t))]
        binned_pos = np.digitize(pos,np.linspace(10,max_d,bins))
        # loop over bins : 
        for bn in np.unique(binned_pos,axis = 0):
            # find bins vectors in same bins : 
            indcs = [all(x) for x in binned_pos == bn] #binned_pos.all(bn)
            vecs_n_bin = vecs_at_t[indcs]
            # consider at least 2 cells in one bin :
            if len(vecs_n_bin) < 2:
                continue
            # sum 
            norm_bin = norm(sum(vecs_n_bin))
            orders.append(norm_bin)
    return orders

In [50]:
rows  = []
deviation_rows = []

for filename in files: 
    # extract tracks : 
    df = pd.read_csv(filename, sep="\t", names=['cell_id', 'time_id', 'time', 'x', 'y', 'z'])   
    df['experiment'] = filename[19:28]
    df['cell_type'] = filename[29:32]
    df['movie'] = filename[38:39]
    ids = df.cell_id.unique()
    tracks = [df[df.cell_id == i][['x','y','z']].to_numpy() for i in ids]
    # filter out super short tracks: 
    tracks = [t for t in tracks if len(t) > 30]
    
    # speed : 
    vec_tracks = np.array([to_vecs(t) for t in tracks])
    speeds = [[norm(v) for v in vec_track] for vec_track in vec_tracks]
    speeds = [item for sublist in speeds for item in sublist]
    speed = np.mean(speeds)
    std_speed = np.std(speeds)
    
    #peristance : 
    autocors = [new_auto(t) for t in tracks]
    half_times = Persist_tracks(autocors)
    ht = np.average(half_times)
    std_ht = np.std(half_times)
    
    #order : 
    ordr1 = Global_order(vec_tracks)
    ordr2,std_ordr2 = OrderAt_T(vec_tracks)
    lcl_ordrs = local_order(tracks)
    lcl_ordr = np.average(lcl_ordrs)
    std_lcl = np.std(lcl_ordrs)
    
    
    rows.append([df.cell_type[0],df.experiment[0],speed,ht,ordr1,ordr2,lcl_ordr])
    deviation_rows.append([df.cell_type[0],df.experiment[0],std_speed,std_ht,std_ordr2,std_lcl])

/home/lau/anaconda2/lib/python3.7/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/lau/anaconda2/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/lau/GIT/FRC_Thesis/CPM_code/OrderNpersist.py:103: RuntimeWarning: invalid value encountered in greater
  max_indeces = np.where(ac > .5)[0]
/home/lau/GIT/FRC_Thesis/CPM_code/OrderNpersist.py:104: RuntimeWarning: invalid value encountered in less
  min_indeces = np.where(ac < .5)[0]
/home/lau/GIT/FRC_Thesis/CPM_code/OrderNpersist.py:108: RuntimeWarning: invalid value encountered in greater
  dt_0 = max(np.where(ac > .5)[0])
/home/lau/GIT/FRC_Thesis/CPM_code/OrderNpersist.py:109: RuntimeWarning: invalid value encountered in less
  dt_1 = min(np.where(ac < .5)[0])


(31, 31, 3)
lenght of order shoulde be min_lenght. Min_lenght 31 == 31
(24, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(15, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(14, 32, 3)
lenght of order shoulde be min_lenght. Min_lenght 32 == 32
(11, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(24, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(10, 31, 3)
lenght of order shoulde be min_lenght. Min_lenght 31 == 31
(23, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(124, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(85, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(11, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(6, 32, 3)
lenght of order shoulde be min_lenght. Min_lenght 32 == 32


/home/lau/anaconda2/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/lau/anaconda2/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/lau/anaconda2/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(15, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(7, 31, 3)
lenght of order shoulde be min_lenght. Min_lenght 31 == 31
(35, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(19, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(45, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30
(10, 30, 3)
lenght of order shoulde be min_lenght. Min_lenght 30 == 30


In [51]:
df1 = pd.DataFrame(data = rows, columns = ['cell_type','experiment', 'speed', 'persistance', 'order', 'order_t','local_order'])

In [52]:
df1

,cell_type,experiment,speed,persistance,order,order_t,local_order
0,cd4,exp090814,2.031867,19.096394,201.993545,15.769745,4.085788
1,cd4,exp090814,2.781124,12.883712,537.891933,21.989994,4.972043
2,cd8,exp090814,2.347425,8.179843,251.023076,14.097144,4.383177
3,cd8,exp090929,2.142296,12.383887,280.785215,13.408530,4.731129
4,cd8,exp090814,3.791209,7.505801,136.688437,16.127611,6.772647
5,cd8,exp090929,1.784613,17.355682,247.631211,12.889660,3.364902
6,cd4,exp101210,3.903979,12.026243,138.852221,15.157344,7.193640
7,cd8,exp101210,4.415100,8.221146,1106.782654,35.191884,8.587819
8,cd8,exp101208,1.763143,16.220981,1645.894122,43.192218,5.721283
9,cd4,exp101208,2.181307,13.706384,989.956507,37.089957,6.732255


In [57]:
#save : 
df1.to_csv('ANLSYS2/Mlab.csv')
df2 = pd.DataFrame(data =deviation_rows, columns = ['cell_type','experiment', 'speed', 'persistance', 'order_t','local_order'])
df2.to_csv('ANLSYS2/Mlab_std.csv')      
           

In [54]:
# print averages : 
mean = df1[['speed','persistance','order','order_t','local_order']].mean()
print("AVERAGE ALL CELL TYPES")
print(mean)
# print(np.average(df1.speed),np.average(df1.persistance),np.average

# Split cd4 / cd8
cd4 = df1[df1.cell_type == 'cd4'][['speed','persistance','order','order_t','local_order']].mean()
print("CD4")
print(cd4)
cd8 = df1[df1.cell_type == 'cd8'][['speed','persistance','order','order_t','local_order']].mean()
print("CD8")
print(cd8)

AVERAGE ALL CELL TYPES
speed            2.571710
persistance     12.689906
order          428.990059
order_t         19.292146
local_order      5.475622
dtype: float64
CD4
speed            2.289712
persistance     12.933704
order          386.510777
order_t         18.904271
local_order      4.903130
dtype: float64
CD8
speed            2.853708
persistance     12.446107
order          471.469341
order_t         19.680020
local_order      5.984503
dtype: float64


In [ ]:
# print averages : 
mean = df1[['speed','persistance','order','order_t']].mean()
print("AVERAGE ALL CELL TYPES")
print(mean)
# print(np.average(df1.speed),np.average(df1.persistance),np.average

# Split cd4 / cd8
cd4 = df1[df1.cell_type == 'cd4'][['speed','persistance','order','order_t']].mean()
print("CD4")
print(cd4)
cd8 = df1[df1.cell_type == 'cd8'][['speed','persistance','order','order_t']].mean()
print("CD8")
print(cd8)

In [ ]:
#         for i,p in enumerate(pos):
#             print(p,binned_pos[i])
#         binned_vecs = sparse_hist(vec_pos,100,bin_index = True)
#         #print(len(binned_vecs.keys()))
#         for key,val in binned_vecs.items():
#             print(key)
#             print(val)
        break
        exit()
        running_vec = 0
#         # loop over bin borders : 
#         for border in bin_border:
#             # loop over vecs at t
#             for tup in vec_pos:
#                 if all(tup[0] )
        
        
#     vec_tracks = np.array([to_vecs(t) for t in tracks])
#     print(vec_tracks.shape)
#     print()
#     # list of order 
#     orders = []
#     for i in range(len(vec_tracks[0])):
#         vecs_at_t = vec_tracks[:,i]
#         # tuple vec wit position : 
#         vec_pos = [(tracks[j][i],vecs_at_t[j]) for j in range(len(vecs_at_t))]
#         vecs_at_t = [t for t in vecs_at_t if norm(t[1]) != 0]
#         pairs = combinations(vec_pos,2)
        
        
#         orders.append(np.average(cosines))
